# Creación y Despliegue de un Sistema RAG con Gemini en AWS SageMaker


Este notebook detalla el proceso completo para construir, desplegar y probar un sistema de **Generación Aumentada por Recuperación (RAG)**. El sistema utiliza el modelo **Gemini de Google** y se despliega como un endpoint en **AWS SageMaker**.


## Flujo de Trabajo

El proceso se divide en los siguientes pasos clave:

1.  **Preparación del Entorno**: Instalación de dependencias y carga de variables de entorno (como la clave de API de Gemini y el nombre del bucket de S3).
2.  **Limpieza de Datos**: Procesamiento y limpieza de documentos de texto fuente para prepararlos para la ingesta.
3.  **Creación del RAG Local**:
    *   Generación de embeddings a partir de los documentos limpios.
    *   Creación de una base de datos vectorial con **ChromaDB**.
    *   Prueba del sistema RAG en el entorno local del notebook para validar la lógica.
4.  **Empaquetado para Despliegue**: Creación de los artefactos necesarios para el endpoint de SageMaker, incluyendo el script de inferencia (`inference.py`) y las dependencias (`requirements.txt`).
5.  **Despliegue en SageMaker**:
    *   Subida de los artefactos del modelo a un bucket de S3.
    *   Creación y despliegue de un **Hugging Face Model** en SageMaker, que actúa como un proxy para realizar llamadas a la API de Gemini.
6.  **Prueba del Endpoint**: Invocación del endpoint desplegado a través de una API Gateway para verificar su funcionamiento en un entorno real.

## Crear RAG

In [42]:

!pip install "transformers==4.38.2" "accelerate==0.27.2" "sentence-transformers==2.5.1" "langchain==0.1.12" "langchain-community==0.0.28" "chromadb>=0.5.0" "pysqlite3-binary" "numpy<2.0" -q

!pip install google-genai -q

!pip install dotenv -q

print("✅ Instalación de dependencias completa.")

✅ Instalación de dependencias completa.


In [ ]:
import os
from dotenv import load_dotenv 

!pip install python-dotenv -q

load_dotenv('var.env') 

# 3. Recuperar las variables para usarlas en el script de despliegue
GEMINI_KEY_VALUE = os.environ.get("GEMINI_API_KEY")
BUCKET_NAME = os.environ.get("S3_BUCKET_NAME")

if not GEMINI_KEY_VALUE:
    raise ValueError("❌ ERROR: La clave GEMINI_API_KEY no se cargó. Revisa tu archivo vars.env.")

print(f"✅ Variables cargadas. Bucket objetivo: {BUCKET_NAME}")

✅ Variables cargadas. Bucket objetivo: sagemaker-us-east-1-891377282708


In [3]:
import os
import re
import html

# --- 1. CONFIGURACIÓN ---
input_folder = 'texts'
output_folder = 'Clean_Text'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# --- 2. FUNCIÓN DE LIMPIEZA V5 (Anti-CSS y Menús) ---
def limpieza_v5_final(texto):
    if not texto: return ""

    # A. PRIMERA PASADA: Limpieza global
    texto = html.unescape(texto)
    texto = re.sub(r'/\*.*?\*/', '', texto, flags=re.DOTALL) # Comentarios /* ... */
    
    lineas = texto.split('\n')
    lineas_limpias = []
    
    lista_negra_exacta = [
        "Skip to content", "Top Menu", "Top Menú", "Main Menu", "MENÚ",
        "Inicio", "UNISON", "DEPARTAMENTO", "FACULTAD",
        "ACERCA DEL PROGRAMA", "INFORMACIÓN PARA ALUMNOS", "ADMISIÓN",
        "DOCENTES", "EDITORIAL", "NOTICIAS Y AVISOS", 
        "NOTICIAS Y AVISOS ANTERIORES", "Previous", "Next",
        "Conócenos", "Misión  y Visión", "Plan de Estudios", "Requisitos",
        "Egreso", "Titulación", "Idioma", "Servicio Social", "CENEVAL",
        "Culturest", "Prácticas  Profesionales", "Programa", "Alumnos",
        "Ingreso", "Plan de Estudios 2025-2", "Plan de Estudios 2005-2",
        "Tesis", "Reestructuración LCC", "Licenciatura en Ciencias de la Computación",
        "AI-Linkup", "Banner Reestructuración LCC", "25 Aniversario LCC",
        "Departamento de Matemáticas", "Universidad de Sonora",
        "Presentación", "Directorio", "Trayectorias Escolares", "LCC-HUB", "Tutorías"
        "-->" 
    ]
    
    # INDICADORES DE CÓDIGO (Si la línea TIENE esto, SE BORRA)
    indicadores_codigo = [
        # HTML/JS
        "body{", "img.emoji", "img.wp-smiley", ".recentcomments", 
        "!function", "window._wpemoji", "var ", "$(document)", "$(\"#", 
        "owlCarousel", "function() {", "});",
        # CSS (Aquí estaba el problema, agregamos las propiedades)
        "!important", "box-shadow:", "height:", "width:", "margin:", 
        "vertical-align:", "padding:", "display:", "border:", "background:",
        ".wp-block-", ".has-"
        "autoPlay:", "items :", "itemsDesktop", "itemsDesktopSmall", "//Set AutoPlay"
    ]

    for linea in lineas:
        linea_strip = linea.strip()
        
        # 1. Vacío
        if not linea_strip:
            continue
        
        # 2. Regla Anti-Código (CSS/JS)
        es_codigo = False
        
        # 2.1 Verificar si contiene palabras prohibidas de código
        for ind in indicadores_codigo:
            if ind in linea_strip: 
                es_codigo = True
                break
        
        # 2.2 Verificar sintaxis técnica (llaves sueltas)
        if "{" in linea_strip and "}" not in linea_strip: 
             if len(linea_strip) < 60: es_codigo = True
        
        if linea_strip == "});" or linea_strip == "}" or linea_strip == "-->":
            es_codigo = True

        if es_codigo:
            continue

        # 3. Regla Lista Negra (Menús y Pies de página)
        es_basura_menu = False
        
        # 3.1 Detección de pie de página largo con barras "|"
        if "Universidad de Sonora" in linea_strip and "|" in linea_strip:
            es_basura_menu = True

        # 3.2 Detección de frases exactas
        if not es_basura_menu:
            for basura in lista_negra_exacta:
                # Coincidencia exacta
                if linea_strip.lower() == basura.lower():
                    es_basura_menu = True
                    break
                # Contenida (para líneas cortas)
                if basura.lower() in linea_strip.lower() and len(linea_strip) < len(basura) + 5:
                    es_basura_menu = True
                    break
        
        if es_basura_menu:
            continue
            
        # 4. Regla de longitud mínima para basura suelta
        #    Borra líneas de menos de 3 letras si no son números o letras
        if len(linea_strip) < 3 and not linea_strip[0].isalnum():
            continue

        # SI SOBREVIVIÓ, GUARDAR
        lineas_limpias.append(linea_strip)

    # C. RECONSTRUCCIÓN Y DETALLES FINALES
    texto_final = "\n".join(lineas_limpias)
    texto_final = re.sub(r'\n{3,}', '\n\n', texto_final) # Maximo 2 enters
    
    return texto_final

# --- 3. EJECUCIÓN ---
try:
    archivos = [f for f in os.listdir(input_folder) if f.endswith('.txt')]
    print(f"🧹 EJECUTANDO LIMPIEZA V5 (Anti-CSS) en {len(archivos)} archivos...")
    print("-" * 40)
    
    count = 0
    for filename in archivos:
        path_origen = os.path.join(input_folder, filename)
        path_destino = os.path.join(output_folder, filename)
        
        try:
            with open(path_origen, 'r', encoding='utf-8', errors='ignore') as f:
                contenido = f.read()
            
            limpio = limpieza_v5_final(contenido)
            
            if len(limpio) > 30:
                with open(path_destino, 'w', encoding='utf-8') as f:
                    f.write(limpio)
                count += 1
            else:
                print(f"⚠️ Archivo quedó vacío: {filename}")
                
        except Exception as e:
            print(f"❌ Error en {filename}: {e}")

    print("-" * 40)
    print(f"🎉 ¡Limpieza terminada! Revisa la carpeta '{output_folder}'.")
    
except FileNotFoundError:
    print("❌ Error: No existe la carpeta 'texts'.")

🧹 EJECUTANDO LIMPIEZA V5 (Anti-CSS) en 39 archivos...
----------------------------------------
----------------------------------------
🎉 ¡Limpieza terminada! Revisa la carpeta 'Clean_Text'.


In [ ]:
import os 
os.environ["GEMINI_API_KEY"] = "tu_clave_aqui"  # Reemplaza con tu clave real

In [ ]:
# --- 1. CONFIGURACIÓN E IMPORTS ---
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import os
import time
from google import genai
from google.genai import types # Importamos types para la configuración
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# --- 2. CARGAR DOCUMENTOS Y EMBEDDINGS ---
print("📂 Cargando Clean_Text y generando Embeddings...")
loader = DirectoryLoader('./Clean_Text', glob="*.txt", loader_cls=TextLoader)
docs_raw = loader.load()

if docs_raw:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
    docs = text_splitter.split_documents(docs_raw)
    
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    db = Chroma.from_documents(docs, embeddings)
    print("✅ Base de Datos ChromaDB lista.")


    try:
        gemini_client = genai.Client()
        MODEL_NAME = 'gemini-2.5-flash'
        print(f"✅ Cliente Gemini {MODEL_NAME} inicializado.")
    except Exception as e:
        print(f"❌ ERROR: No se pudo iniciar el cliente Gemini. Revisa tu clave API.")
        raise e

    # --- 4. FUNCIÓN DE PREGUNTA (USANDO GEMINI) ---
    def preguntar_gemini(pregunta_usuario):
        start_time = time.time()
        print(f"\n❓ {pregunta_usuario}")
        print("🔍 Buscando contexto en ChromaDB...")
        
        docs_encontrados = db.similarity_search(pregunta_usuario, k=7)
        contexto_acumulado = ""
        
        for doc in docs_encontrados:
            nombre = doc.metadata.get('source', 'unknown').split('/')[-1]
            clean_content = doc.page_content.replace('\n', ' ').strip()
            contexto_acumulado += f"- [Archivo: {nombre}] {clean_content}\n\n"

        # Prompt con reglas para la API de Gemini
        prompt_final = f"""<|im_start|>system
Eres un asistente administrativo útil. Responde de forma muy concisa usando SOLO el siguiente contexto.
Reglas:
1. Responde usando SOLO el contexto.
3. Si la respuesta no está, di "No tengo información."
<|im_end|>
<|im_start|>user
Contexto:\n{contexto_acumulado}\n\nPregunta: {pregunta_usuario}
<|im_end|>
<|im_start|>assistant
"""
        
        try:
            # Generar Respuesta con la API de Gemini
            response = gemini_client.models.generate_content(
                model=MODEL_NAME,
                contents=[prompt_final],
                config=types.GenerateContentConfig(temperature=0.01)
            )

            respuesta_limpia = response.text.strip()
            end_time = time.time()
            
            print("💡 R:", respuesta_limpia)
            print(f"⏱️ Tiempo total RAG (Contexto + API): {end_time - start_time:.2f} segundos")
            print("-" * 60)

        except Exception as e:
            print(f"❌ Error API: {e}")
            
    # --- 5. PRUEBAS ---
    preguntar_gemini("¿Quién es el responsable de tutorías?")


else:
    print("⚠️ Carpeta vacía.")

📂 Cargando Clean_Text y generando Embeddings...
✅ Base de Datos ChromaDB lista.
✅ Cliente Gemini gemini-2.5-flash inicializado.

❓ ¿Quién es el responsable de tutorías?
🔍 Buscando contexto en ChromaDB...
💡 R: Dr. Edelmira Rodríguez Alcantar.
⏱️ Tiempo total RAG (Contexto + API): 1.41 segundos
------------------------------------------------------------


## Subir al EndPoint

In [57]:
import os

# Define la ruta base para el despliegue
DEPLOY_DIR = 'model_deploy/code'

# Elimina el directorio anterior si existe y crea uno nuevo limpio
!rm -rf model_deploy
!mkdir -p {DEPLOY_DIR}

print(f"✅ Directorio de trabajo creado: {DEPLOY_DIR}")

✅ Directorio de trabajo creado: model_deploy/code


In [58]:
%%writefile model_deploy/code/requirements.txt
transformers==4.38.2
accelerate==0.27.2
sentence-transformers==2.5.1
langchain==0.1.12
langchain-community==0.0.28
chromadb>=0.5.0
pysqlite3-binary
numpy<2.0
google-genai

Writing model_deploy/code/requirements.txt


In [ ]:
%%writefile model_deploy/code/inference.py
import os
import json
import boto3
import zipfile
import shutil
import time 
from google import genai
from google.genai import types 

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY') 
BUCKET_NAME = os.environ.get('S3_BUCKET', 'sagemaker-us-east-1-891377282708')
DB_ZIP_KEY = os.environ.get('DB_ZIP_KEY', 'rag-artifacts/chroma_db.zip')
EXTRACT_PATH = '/tmp/chroma_db'
MODEL_NAME = 'gemini-2.5-flash' # Modelo rápido

db_client = None 
gemini_client = None

def model_fn(model_dir):
    """Inicializa la DB ChromaDB y el cliente Gemini (se ejecuta 1 vez)."""
    global db_client, gemini_client
    print("🚀 [Start] Iniciando Proxy Gemini en SageMaker...")
    
    if not GEMINI_API_KEY:
        raise EnvironmentError("GEMINI_API_KEY no configurada. ¡Revisa las variables de entorno del despliegue!")

    # 1. Descargar y Cargar DB Chroma
    s3_client = boto3.client('s3')
    local_zip_path = '/tmp/chroma_db.zip'

    if os.path.exists(EXTRACT_PATH):
        shutil.rmtree(EXTRACT_PATH)
    os.makedirs(EXTRACT_PATH, exist_ok=True)
    
    s3_client.download_file(BUCKET_NAME, DB_ZIP_KEY, local_zip_path)
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
    
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    db_client = Chroma(persist_directory=EXTRACT_PATH, embedding_function=embeddings)
    
    # 2. Inicializar Cliente Gemini
    gemini_client = genai.Client(api_key=GEMINI_API_KEY)
    
    print("✅ [Ready] Sistema RAG con Proxy Gemini listo.")
    return {"db": db_client, "gemini_client": gemini_client}


def predict_fn(data, context):
    """Ejecuta la búsqueda RAG y llama a Gemini (se ejecuta en cada invocación)."""
    
    # 1. Recuperar artefactos
    db = context["db"]
    gemini = context["gemini_client"]

    # 2. Leer la pregunta del payload
    if isinstance(data, list):
        input_data = data[0]
    else:
        input_data = data
        
    # 'inputs' es la clave que envía tu Lambda
    pregunta = input_data.get('inputs', input_data.get('question', '')) 
    
    if not pregunta:
        raise ValueError("Pregunta vacía recibida.")
        
    # 3. RAG: Búsqueda de contexto (K=3, para velocidad)
    docs_encontrados = db.similarity_search(pregunta, k=3)
    contexto_acumulado = "\n".join([doc.page_content.replace('\n', ' ').strip() for doc in docs_encontrados])
    
    # 4. Prompt para Gemini
    prompt_final = f"""<|im_start|>system
Eres un asistente administrativo útil. Responde de forma muy concisa usando SOLO el siguiente contexto.
Reglas:
1. Responde usando SOLO el contexto.
3. Si la respuesta no está, di "No tengo información."
<|im_end|>
<|im_start|>user
Contexto:\n{contexto_acumulado}\n\nPregunta: {pregunta}
<|im_end|>
<|im_start|>assistant
"""
    
    response = gemini.models.generate_content(
        model=MODEL_NAME,
        contents=[prompt_final],
        config=types.GenerateContentConfig(temperature=0.01)
    )
    
    texto_respuesta = response.text.strip()
    
    # 6. Devolver en el formato esperado por la Lambda (lista de dicts con 'generated_text')
    return [{"generated_text": texto_respuesta}]

Writing model_deploy/code/inference.py


In [60]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
import os
import time
import json
from dotenv import load_dotenv # Necesario si no se ejecutó antes

# --- 1. CARGAR VARIABLES (USANDO os.environ) ---
# Asegúrate de que tu archivo 'vars.env' esté en el mismo directorio.
if 'GEMINI_API_KEY' not in os.environ:
    load_dotenv('vars.env')

GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY") 
BUCKET_NAME = os.environ.get("S3_BUCKET_NAME", 'sagemaker-us-east-1-891377282708')

if not GEMINI_API_KEY:
    raise ValueError("❌ ERROR: La clave GEMINI_API_KEY no se cargó. Crea y verifica tu archivo vars.env.")

ENDPOINT_NAME = f'gemini-proxy-{time.strftime("%Y%m%d-%H%M%S")}'

# --- 2. EMPAQUETAR Y SUBIR ---
print("📦 Creando paquete final del Proxy Gemini...")

# Nota: Asegúrate de que las celdas %%writefile hayan creado la carpeta model_deplo/code
!cd model_deploy && tar -czvf ../model_proxy.tar.gz code

# Subir el paquete a S3
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
model_uri = sess.upload_data(path='model_proxy.tar.gz', bucket=BUCKET_NAME, key_prefix='rag-code-proxy')

print(f"✅ Paquete de código subido a: {model_uri}")


# --- 3. DESPLEGAR ENDPOINT PROXY ---
huggingface_model = HuggingFaceModel(
    model_data=model_uri,
    role=role,
    # Estos valores son necesarios para el contenedor base HuggingFace
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310",
    # CRÍTICO: INYECTAR LAS VARIABLES DE ENTORNO
    env={ 
        'GEMINI_API_KEY': GEMINI_API_KEY, # La clave se inyecta de forma segura
        'S3_BUCKET': BUCKET_NAME,
        'DB_ZIP_KEY': 'rag-artifacts/chroma_db.zip',
        'HF_TASK': 'text-generation',
        'SAGEMAKER_MODEL_SERVER_WORKERS': '1',
    }
)

print("🚀 LANZANDO ENDPOINT PROXY...")
try:
    predictor = huggingface_model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.xlarge", 
        endpoint_name=ENDPOINT_NAME
    )
    print("--------------------------------------------------")
    print(f"✅ ¡LISTO! Tu nuevo Endpoint Proxy es: {predictor.endpoint_name}")
    print("--------------------------------------------------")
except Exception as e:
    print(f"❌ Error al desplegar el Endpoint: {e}")

📦 Creando paquete final del Proxy Gemini...
code/
code/requirements.txt
code/inference.py
✅ Paquete de código subido a: s3://sagemaker-us-east-1-891377282708/rag-code-proxy/model_proxy.tar.gz
🚀 LANZANDO ENDPOINT PROXY...
--------!--------------------------------------------------
✅ ¡LISTO! Tu nuevo Endpoint Proxy es: gemini-proxy-20251206-231118
--------------------------------------------------


# Llamar Lambda para probar funcionamiento del endpoint y RAG

In [37]:
import time
import json
import requests
from requests.exceptions import Timeout

# --- CONFIGURACIÓN ---
API_GATEWAY_URL = "https://7cfazo47r0.execute-api.us-east-1.amazonaws.com/default/RAG_Backend"

# El timeout de requests.post es el tiempo que el cliente esperará (ponemos 29s por si acaso)
CLIENT_TIMEOUT = 29 

# Pregunta compleja que fallaba antes
pregunta_final = "¿Quién es el responsable de tutorías y cuáles son todos los requisitos de idioma para titulación?"

# Payload de la Web (usa 'question' o 'inputs')
payload = {
    "question": pregunta_final
}

print(f"📡 Probando API Gateway con Proxy Gemini: {API_GATEWAY_URL}")
print(f"⏳ Pregunta: '{pregunta_final}'")
print("-" * 60)

start_time = time.time()

try:
    # Hacemos la petición como si fuéramos la página web
    response = requests.post(
        API_GATEWAY_URL, 
        json=payload, 
        timeout=CLIENT_TIMEOUT
    )
    
    end_time = time.time()
    
    # Intenta decodificar el JSON de la respuesta
    try:
        result = response.json()
    except json.JSONDecodeError:
        result = {"answer": f"Respuesta no JSON: {response.text[:100]}..."}

    if response.status_code == 200:
        print(f"✅ ¡ÉXITO! Respuesta en {end_time - start_time:.2f} segundos.")
        print("--------------------------------------------------")
        print(f"Respuesta Final:\n{result.get('answer', 'Respuesta inesperada')}")
    else:
        print(f"❌ FALLO DE SERVIDOR/PROXY: Status Code {response.status_code}")
        print(f"Detalle del error: {result.get('error', 'Error no detallado.')}")
        
except Timeout:
    # Este error solo debería ocurrir si todo el proceso tarda > 29s, lo cual ya no debería pasar
    print(f"❌ ERROR: Timeout (Límite de 29s alcanzado). El Proxy de Gemini Falló o el Endpoint no está activo.")
except Exception as e:
    print(f"❌ ERROR DE CONEXIÓN: {e}")

📡 Probando API Gateway con Proxy Gemini: https://7cfazo47r0.execute-api.us-east-1.amazonaws.com/default/RAG_Backend
⏳ Pregunta: '¿Quién es el responsable de tutorías y cuáles son todos los requisitos de idioma para titulación?'
------------------------------------------------------------
✅ ¡ÉXITO! Respuesta en 3.41 segundos.
--------------------------------------------------
Respuesta Final:
El responsable del programa de tutorías en la Licenciatura en Ciencias de la Computación es la Dra. Edelmira Rodríguez Alcantar.

Los requisitos de idioma para titulación son:
*   Acreditar el nivel IV de inglés en el Departamento de Lenguas Extranjeras.
*   Acreditar el curso Comprensión de Lectura 1 del Departamento de Lenguas Extranjeras.
*   Acreditar la obtención de al menos 320 puntos en el examen TOEFL.
*   Acreditar una estancia internacional en idioma inglés de tres meses como mínimo.
*   Acreditar estudios escolarizados concluidos, realizados en idioma inglés, equivalentes a los niveles d